# 🧙‍♂️ Gandalf RAG Chatbot with Mistral-7B (Fully Local)

In [4]:
# 📦 Install required packages (uncomment if needed)
# !pip install llama-cpp-python langchain langchain-community


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/67.3 MB ? eta -:--:--
     - -------------------------------------- 2.4/67.3 MB 14.5 MB/s eta 0:00:05
     --- ------------------------------------ 5.5/67.3 MB 14.8 MB/s eta 0:00:05
     ----- ---------------------------------- 8.7/67.3 MB 14.9 MB/s eta 0:00:04
     ------ -------------------------------- 11.8/67.3 MB 14.9 MB/s eta 0:00:04
     -------- ------------------------------ 14.9/67.3 MB 14.9 MB/s eta 0:00:04
     ---------- ---------------------------- 18.4/67.3 MB 14.9 MB/s eta 0:00:04
     ------------ -------------------------- 21.2/67.3 MB 14.8 MB/s eta 0:00:04
     ------------- ------------------------- 23.9/67.3 MB 14.6 MB/s eta 0:00:03
     --------------- ----------------------- 27.0/67.3 MB 14.6 MB/s eta 0:00:03
     ----------------- --------------------- 30.1/67.3 MB 14.7 MB/s eta 0:00:03
     ------------------- ------------------- 33.3/

## 🔧 Load Embeddings & FAISS Vectorstore

In [6]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from dotenv import load_dotenv
import os
from warnings import filterwarnings

load_dotenv()

# Load the same embedding model used to create the index
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Load the vectorstore from disk
vectorstore = FAISS.load_local(
    "gandalf_index",
    embeddings,
    allow_dangerous_deserialization=True
)
retriever = vectorstore.as_retriever()


## 🧠 Load Mistral-7B-Instruct Locally with llama-cpp

In [11]:
from langchain_community.llms import LlamaCpp

model_path = "models/mistral/mistral-7b-instruct-v0.1.Q4_K_M.gguf"

llm = LlamaCpp(
    model_path=model_path,
    n_ctx=2048,
    temperature=0.7,
    max_tokens=256,
    verbose=True
)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from models/mistral/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:            

In [14]:
!pip uninstall llama-cpp-python -y
!pip install llama-cpp-python --force-reinstall --upgrade


Found existing installation: llama_cpp_python 0.3.8
Uninstalling llama_cpp_python-0.3.8:
  Successfully uninstalled llama_cpp_python-0.3.8


You can safely remove it manually.


Defaulting to user installation because normal site-packages is not writeable
  Using cached llama_cpp_python-0.3.8-cp313-cp313-win_amd64.whl
  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached MarkupSafe-3.0.2-cp313-cp313-win_amd64.whl.metadata (4.1 kB)
Using cached diskcache-5.6.3-py3-none-any.whl (45 kB)
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ------- -------------------------------- 2.4/12.6 MB 14.4 MB/s eta 0:00:01
   ------------------ --------------------- 5.8/12.6 MB 14.7 MB/s eta 0:00:01
   --------------------------- ------------ 8.7/12.6 MB 14.8 MB/s eta 0:00:01
   -------------------------------------- - 12.1/12.6 MB 14.9 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 14.4 MB/s eta 0:00:00
Using cached MarkupSafe-3.0.2-cp313-cp313-win_amd64.whl (15 kB)
  Attempting uninstall

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.


## 🔁 Ask Questions (RAG)

In [18]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

question = "who is smaegal?"
result = qa_chain.invoke({"query": question})

print("🧙 Gandalf says:\n", result['result'])

Llama.generate: 45 prefix-match hit, remaining 972 prompt tokens to eval
llama_perf_context_print:        load time =   45821.64 ms
llama_perf_context_print: prompt eval time =   30165.33 ms /   972 tokens (   31.03 ms per token,    32.22 tokens per second)
llama_perf_context_print:        eval time =    6357.05 ms /    68 runs   (   93.49 ms per token,    10.70 tokens per second)
llama_perf_context_print:       total time =   36567.35 ms /  1040 tokens


🧙 Gandalf says:
  Sme´agol is a hobbit. He is the most curious-minded member of his family, and he is interested in roots and beginnings. He has a friend named De´agol who shares similar interests. They go on an adventure to the Gladden Fields where they encounter a great fish while fishing.


In [ ]:
# gandalf_chat.py

from langchain_community.llms import LlamaCpp
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
import os
from dotenv import load_dotenv

# ---- Load environment variables
load_dotenv()

# ---- Load embeddings (must match the one used to build FAISS index)
print("🔍 Loading embeddings...")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# ---- Load FAISS vectorstore
print("📚 Loading vectorstore index...")
retriever = FAISS.load_local(
    "gandalf_index",
    embeddings,
    allow_dangerous_deserialization=True
).as_retriever()

# ---- Load Mistral GGUF model locally using llama-cpp-python
print("🧠 Loading Mistral model locally...")
llm = LlamaCpp(
    model_path="models/mistral/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    n_ctx=2048,
    temperature=0.7,
    max_tokens=512,
    verbose=False
)

# ---- Setup RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

# ---- Run a loop for chat
print("\n🧙 Gandalf is ready to answer your questions.\n(Type 'exit' to quit)\n")

while True:
    user_input = input("🗨️  Ask Gandalf: ")
    if user_input.strip().lower() in ("exit", "quit"):
        print("👋 Farewell, traveler.")
        break

    result = qa_chain.invoke({"query": user_input})
    print("\n🧙 Gandalf says:\n", result['result'], "\n")
